# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 13 2023 2:04PM,257276,10,Yusen-9154036,Yusen – 3D Matrix,Released
1,Feb 13 2023 1:44PM,257275,10,Yusen-9154007,Yusen – 3D Matrix,Released
2,Feb 13 2023 1:44PM,257275,10,Yusen-9154009,Yusen – 3D Matrix,Released
3,Feb 13 2023 1:29PM,257273,10,Eme-108785,Emerginnova,Released
4,Feb 13 2023 1:29PM,257273,10,Enova-11350,Emerginnova,Released
5,Feb 13 2023 1:28PM,257272,10,0086377018,ISDIN Corporation,Released
6,Feb 13 2023 1:28PM,257272,10,0086377046,ISDIN Corporation,Released
7,Feb 13 2023 1:28PM,257272,10,0086377114,ISDIN Corporation,Released
8,Feb 13 2023 1:28PM,257272,10,0086377112,ISDIN Corporation,Released
9,Feb 13 2023 1:28PM,257272,10,0086377113,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,257270,Released,15
47,257272,Released,5
48,257273,Released,2
49,257275,Released,2
50,257276,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257270,NaN,NaN,15.0
257272,NaN,NaN,5.0
257273,NaN,NaN,2.0
257275,NaN,NaN,2.0
257276,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257186,1.0,2.0,0.0
257194,40.0,0.0,1.0
257198,0.0,1.0,0.0
257199,0.0,0.0,8.0
257200,0.0,0.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257186,1,2,0
257194,40,0,1
257198,0,1,0
257199,0,0,8
257200,0,0,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257186,1,2,0
1,257194,40,0,1
2,257198,0,1,0
3,257199,0,0,8
4,257200,0,0,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257186,1,2,
1,257194,40,,1
2,257198,,1,
3,257199,,,8
4,257200,,,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 13 2023 2:04PM,257276,10,Yusen – 3D Matrix
1,Feb 13 2023 1:44PM,257275,10,Yusen – 3D Matrix
3,Feb 13 2023 1:29PM,257273,10,Emerginnova
5,Feb 13 2023 1:28PM,257272,10,ISDIN Corporation
10,Feb 13 2023 1:24PM,257270,10,"Methapharm, Inc."
25,Feb 13 2023 1:14PM,257269,10,"Methapharm, Inc."
28,Feb 13 2023 1:04PM,257268,10,Ivaoes Animal Health
29,Feb 13 2023 12:38PM,257265,10,Bio-PRF
38,Feb 13 2023 12:37PM,257266,21,"NBTY Global, Inc."
39,Feb 13 2023 12:25PM,257261,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 13 2023 2:04PM,257276,10,Yusen – 3D Matrix,,,1
1,Feb 13 2023 1:44PM,257275,10,Yusen – 3D Matrix,,,2
2,Feb 13 2023 1:29PM,257273,10,Emerginnova,,,2
3,Feb 13 2023 1:28PM,257272,10,ISDIN Corporation,,,5
4,Feb 13 2023 1:24PM,257270,10,"Methapharm, Inc.",,,15
5,Feb 13 2023 1:14PM,257269,10,"Methapharm, Inc.",,,3
6,Feb 13 2023 1:04PM,257268,10,Ivaoes Animal Health,,,1
7,Feb 13 2023 12:38PM,257265,10,Bio-PRF,,,9
8,Feb 13 2023 12:37PM,257266,21,"NBTY Global, Inc.",,,1
9,Feb 13 2023 12:25PM,257261,10,Bio-PRF,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 2:04PM,257276,10,Yusen – 3D Matrix,1,,
1,Feb 13 2023 1:44PM,257275,10,Yusen – 3D Matrix,2,,
2,Feb 13 2023 1:29PM,257273,10,Emerginnova,2,,
3,Feb 13 2023 1:28PM,257272,10,ISDIN Corporation,5,,
4,Feb 13 2023 1:24PM,257270,10,"Methapharm, Inc.",15,,
5,Feb 13 2023 1:14PM,257269,10,"Methapharm, Inc.",3,,
6,Feb 13 2023 1:04PM,257268,10,Ivaoes Animal Health,1,,
7,Feb 13 2023 12:38PM,257265,10,Bio-PRF,9,,
8,Feb 13 2023 12:37PM,257266,21,"NBTY Global, Inc.",1,,
9,Feb 13 2023 12:25PM,257261,10,Bio-PRF,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 2:04PM,257276,10,Yusen – 3D Matrix,1,,
1,Feb 13 2023 1:44PM,257275,10,Yusen – 3D Matrix,2,,
2,Feb 13 2023 1:29PM,257273,10,Emerginnova,2,,
3,Feb 13 2023 1:28PM,257272,10,ISDIN Corporation,5,,
4,Feb 13 2023 1:24PM,257270,10,"Methapharm, Inc.",15,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 2:04PM,257276,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,Feb 13 2023 1:44PM,257275,10,Yusen – 3D Matrix,2.0,NaN,NaN
2,Feb 13 2023 1:29PM,257273,10,Emerginnova,2.0,NaN,NaN
3,Feb 13 2023 1:28PM,257272,10,ISDIN Corporation,5.0,NaN,NaN
4,Feb 13 2023 1:24PM,257270,10,"Methapharm, Inc.",15.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 2:04PM,257276,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,Feb 13 2023 1:44PM,257275,10,Yusen – 3D Matrix,2.0,0.0,0.0
2,Feb 13 2023 1:29PM,257273,10,Emerginnova,2.0,0.0,0.0
3,Feb 13 2023 1:28PM,257272,10,ISDIN Corporation,5.0,0.0,0.0
4,Feb 13 2023 1:24PM,257270,10,"Methapharm, Inc.",15.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7717204,313.0,4.0,1.0
12,514469,0.0,2.0,0.0
15,771677,20.0,72.0,11.0
19,1543399,32.0,1.0,40.0
21,514487,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7717204,313.0,4.0,1.0
1,12,514469,0.0,2.0,0.0
2,15,771677,20.0,72.0,11.0
3,19,1543399,32.0,1.0,40.0
4,21,514487,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,313.0,4.0,1.0
1,12,0.0,2.0,0.0
2,15,20.0,72.0,11.0
3,19,32.0,1.0,40.0
4,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,313.0
1,12,Released,0.0
2,15,Released,20.0
3,19,Released,32.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Completed,1.0,0.0,11.0,40.0,0.0
Executing,4.0,2.0,72.0,1.0,0.0
Released,313.0,0.0,20.0,32.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Completed,1.0,0.0,11.0,40.0,0.0
1,Executing,4.0,2.0,72.0,1.0,0.0
2,Released,313.0,0.0,20.0,32.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Completed,1.0,0.0,11.0,40.0,0.0
1,Executing,4.0,2.0,72.0,1.0,0.0
2,Released,313.0,0.0,20.0,32.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()